In [1]:
'''
# Purpose: To replace the fields and convert to the reference grid
# History: 24/03/2023 v1
# -------------------------------------
# Note:
#       You can find how to run this script in the comments. At the moment, 
#   it can only handle single level and pseudo level fields. It cannot be 
#   used to replace model level fields. If you need to do so, we can add more functions to it later.
#       I am now using scipy.interpolate.RegularGridInterpolator to regrid the field. 
#   However the results are different from those using cdo. So please check again to see 
#   if you can use /g/data/p66/lz3062/replace_dumpfile/replace_fields/bi889a.da10500101_00_modified
#   to run the experiments.
#       Here we test the LAI, and other CNP related fields for CAABLE initial to use.
#   Those fields in bi889a.da10500101_00 were wrong so that need to be replaced from ESM1.5 start dump.
# -------------------------------------
# Generate by Lynn
# Lynn Zhou <Linjing.Zhou@bom.gov.au>
'''
import numpy as np
from scipy.interpolate import RegularGridInterpolator
import mule
import warnings
warnings.filterwarnings('ignore')
src_file = '/g/data/p66/lz3062/replace_dumpfile/restart_dump.astart.esm1.5'
trg_file = '/g/data/p66/lz3062/replace_dumpfile/bi889a.da10500101_00'
ofile = '/g/data/p66/lz3062/replace_dumpfile/delete_fields/bi889a.da10500101_00_modified'
stash_list = [217,875,880,881,883,884,885,887,888]

In [2]:
def get_grid(umf,stash):
    for ifield, field in enumerate(umf.fields):
        if field.lbuser4 == stash:
            nlat,nlon = field.get_data().shape
            lat_s = field.bzy+field.bdy
            dlat = field.bdy
            lon_s = field.bzx+field.bdx
            dlon = field.bdx
            break
    lon = np.arange(lon_s,lon_s+dlon*nlon,dlon)
    lat = np.arange(lat_s,lat_s+dlat*nlat,dlat)
    return lon,lat

class replaceOperator(mule.DataOperator):
    """Operator which replace data with delta"""
    def __init__(self,delta):
        self.delta = delta
    def new_field(self, source_field):
        """Creates the new field object"""
        field = source_field.copy()
        return field
    def transform(self, source_field, new_field):
        """Performs the data manipulation"""
        data = source_field.get_data()
        data_out = self.delta
        return data_out
    
def replace(stash,src,trg):
    print(f'=========================================================================================')
    print(f' Replacing stash code {stash}')
    print(f'=========================================================================================')
    for src_ifield, src_field in enumerate(src.fields):
        if src_field.lbuser4 == stash:
            data = np.where(src_field.get_data()==-1073741824,np.nan,src_field.get_data())
            interp = RegularGridInterpolator((lat_src,lon_src),data,bounds_error=False,fill_value=None)
            data_int = interp((lats_trg,lons_trg))
            for trg_ifield,trg_field in enumerate(trg.fields):
                if trg_field.lbuser4 == src_field.lbuser4 and trg_field.lbuser5 == src_field.lbuser5:
                    print(f'[Info] Field {trg_ifield} in the target file matches field {src_ifield} in the source file. Replacing...')
                    replace_operator = replaceOperator(data_int)
                    trg.fields[trg_ifield] = replace_operator(trg_field)
                    print('[Info] Replaced.')
    return trg

In [3]:
src = mule.DumpFile.from_file(src_file)
trg = mule.DumpFile.from_file(trg_file)

print(f'Target file: {trg_file}')
print(f'Source file {src_file}')
print(f'=========================================================================================')
print(f' Removing stash code 151, 152 and 153')
print(f'=========================================================================================')
print('[Info] Removing...')
for field in list(trg.fields):
    if field.lbuser4 in [151,152,153]:
        trg.fields.remove(field)
print('[Info] Removed.')

for stash in stash_list:
    lon_src,lat_src = get_grid(src,stash)
    lon_trg,lat_trg = get_grid(trg,stash)
    lats_trg,lons_trg = np.meshgrid(lat_trg,lon_trg,indexing='ij')
    trg = replace(stash,src,trg)
    
print(f'=========================================================================================')
print(f' Write to file')
print(f'=========================================================================================')
print('[Info] Writing...')
trg.to_file(ofile)
print('[Info] Done.')

Target file: /g/data/p66/lz3062/replace_dumpfile/bi889a.da10500101_00
Source file /g/data/p66/lz3062/replace_dumpfile/restart_dump.astart.esm1.5
 Removing stash code 151, 152 and 153
[Info] Removing...
[Info] Removed.
 Replacing stash code 217
[Info] Field 973 in the target file matches field 1214 in the source file. Replacing...
[Info] Replaced.
[Info] Field 974 in the target file matches field 1215 in the source file. Replacing...
[Info] Replaced.
[Info] Field 975 in the target file matches field 1216 in the source file. Replacing...
[Info] Replaced.
[Info] Field 976 in the target file matches field 1217 in the source file. Replacing...
[Info] Replaced.
[Info] Field 977 in the target file matches field 1218 in the source file. Replacing...
[Info] Replaced.
[Info] Field 978 in the target file matches field 1219 in the source file. Replacing...
[Info] Replaced.
[Info] Field 979 in the target file matches field 1220 in the source file. Replacing...
[Info] Replaced.
[Info] Field 980 in t